Import required python libraries

In [3]:
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from datetime import timedelta

In [4]:
wdir = '/path/to/your/working/directory/'

  Loop through each intervened years 
  
  *Make sure the input folder does not contain any additional csv files with similar naming conventions*
  

In [1]:
for year in range(2013, 2020):
    f1 = wdir+str(year)+'_chirps.csv'
    print('Processing year...',year)
    
    # check to see if the path/file exists
    if os.path.exists(f1):
        ch = pd.read_csv(wdir+str(year)+'_chirps.csv')
        l7 = pd.read_csv(wdir+str(year)+'_L7.csv')
        l8 = pd.read_csv(wdir+str(year)+'_L8.csv')
        
        # set date as index
        ch = ch.set_index('id')
        l7 = l7.set_index('Date')
        l8 = l8.set_index('Date')
        
        #convert index to datetime object
        ch.index = pd.to_datetime(ch.index)
        l7.index = pd.to_datetime(l7.index)
        l8.index = pd.to_datetime(l8.index)
        
        # selecting only months Jul-Nov for precip
        ch1 = ch.loc[ch.index.month>6]
        ch1 = ch1.loc[ch1.index.month<10]
        
        # aggregating precipitatio data to annual scale and resent index
        chy = ch1.resample('Y').sum()
        #chy.index = chy.index-timedelta(360)
        
        # compute precip anomaly using long term mean and std. deviation
        ch_anom = (chy - chy.mean())/chy.std()
        
        #loop through each of the columns (subpolygons) intervened this particular year
        cols = ch.columns
        for col in cols:    
            df = pd.DataFrame()
            df['Date'] = pd.date_range('1/1/2002','12/31/2020')
            df = df.set_index('Date')
            df = df.merge(l7[col],left_index=True, right_index=True,how='left')
            df = df.merge(l8[col],left_index=True, right_index=True,how='left')
            df.columns = ['L7','L8']

            # if interested in monthly data, use this section and comment out yearly one
            '''
            # Monthly composite
            df2 = df.resmaple('M').max()
            '''
            
            # yearly composite at 95th precentile 
            df2 = df.resample('Y').quantile(.95)
            
            
            #--------------
            # plotting data
            #---------------
            
            fig, ax = plt.subplots(constrained_layout=True, figsize = (9,5))
            
            # plot NDVI data
            plt.plot(df2.L8, label = 'L8', ls= '-', marker = 's')
            plt.plot(df2.L7, label = 'L7',  ls= '-',marker = 'o')
            plt.ylabel('NDVI')
            plt.ylim(-.1,.5)
            plt.plot([pd.to_datetime(year,format='%Y'),pd.to_datetime(year,format='%Y')],[-.5,5], ls = '--', color = 'gray')
            plt.legend(loc = 4)
            plt.title(col+'  '+str(year))
            
            # Plot precipitation data
            ax2 = ax.twinx()
            ax2.bar(ch_anom.index,ch_anom[col], width = 50, color = 'black')
            ax2.set_ylabel('Precip Anomalies')
            ax2.plot(ch_anom.index,np.repeat(0,len(ch_anom.index)), ls = '--', color = 'black')
            
            plt.show()
            exit
            
            plt.savefig(wdir+str(year)+'_'+col+'.png',dpi = 500)
            plt.close()


NameError: name 'wdir' is not defined